## 1. Установка библиотек

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
import time
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import plotly.express as px

## 2. Работа с датасетом Ирис

### 2.1 Установка датасета

In [65]:
iris = load_iris()
X = iris.data
y = iris.target
feature_names = iris.feature_names
target_names = iris.target_names

### 2.2 Анализ формата

In [66]:
features_matrix = X
target_vector = y

print("Размер матрицы признаков:", features_matrix.shape)
print("Размер вектора целевых переменных:", target_vector.shape)

print("\nЗанимаемая память:")
print(f"Матрица признаков: {features_matrix.nbytes}")
print(f"Вектор целевых переменных: {target_vector.nbytes} байт")

Размер матрицы признаков: (150, 4)
Размер вектора целевых переменных: (150,)

Занимаемая память:
Матрица признаков: 4800
Вектор целевых переменных: 1200 байт


### 2.4 Нормализация

In [67]:
def normalize_data(data):
    min_vals = data.min(axis=0)
    max_vals = data.max(axis=0)
    normalized = (data - min_vals) / (max_vals - min_vals)
    return normalized

X_normalized = normalize_data(features_matrix)

print("До нормализации (первые пять строк):")
print(features_matrix[:5])
print("После нормализации:")
print(X_normalized[:5])

До нормализации (первые пять строк):
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
После нормализации:
[[0.22222222 0.625      0.06779661 0.04166667]
 [0.16666667 0.41666667 0.06779661 0.04166667]
 [0.11111111 0.5        0.05084746 0.04166667]
 [0.08333333 0.45833333 0.08474576 0.04166667]
 [0.19444444 0.66666667 0.06779661 0.04166667]]


### 2.5 Создание категории с использованием заданных квантилей

In [68]:
sepal_length = X_normalized[:, 0]

def create_categories(data):
    small_threshold = np.quantile(data, 0.25)
    large_threshold = np.quantile(data, 0.75)

    categories = []
    for value in data:
        if value < small_threshold:
            categories.append('small')
        elif value > large_threshold:
            categories.append('big')
        else:
            categories.append('medium')
    
    return categories

sepal_categories = create_categories(sepal_length)

print(f"\nРаспределение категорий:")
unique, counts = np.unique(sepal_categories, return_counts=True)
for category, count in zip(unique, counts):
    print(f"{category}: {count} объекта(ов)")


Распределение категорий:
big: 35 объекта(ов)
medium: 83 объекта(ов)
small: 32 объекта(ов)


### 2.6 Разделение датасета

In [69]:
def split_dataset(X, y, test_size=0.2, random_seed=14):
    np.random.seed(random_seed)
    n_samples = X.shape[0]
    n_test = int(n_samples * test_size)

    indices = np.random.permutation(n_samples)

    test_indices = indices[:n_test]
    train_indices = indices[n_test:]

    X_train = X[train_indices]
    X_test = X[test_indices]
    y_train = y[train_indices]
    y_test = y[test_indices]

    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = split_dataset(X_normalized, target_vector)

print("Размеры выборок после разделения:")
print(f"Тренировочная выборка: {X_train.shape}")
print(f"Тестовая выборка: {X_test.shape}")
print(f"Тренировочные метки: {y_train.shape}")
print(f"Тестовые метки: {y_test.shape}")

Размеры выборок после разделения:
Тренировочная выборка: (120, 4)
Тестовая выборка: (30, 4)
Тренировочные метки: (120,)
Тестовые метки: (30,)


## 3. Работа с методом классификации SVC

### 3.1 Сравнение зависимости метода классификации от гиперпараметров и данных

#### 3.1.1 Функция для замера основных параметров тренировки и предсказания

In [ ]:
X_train_orig, X_test_orig, y_train_orig, y_test_orig = split_dataset(
    features_matrix, target_vector
)

def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    start_time = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start_time

    start_time = time.time()
    y_pred = model.predict(X_test)
    predict_time = time.time() - start_time

    accuracy = accuracy_score(y_test, y_pred)
    print(f"{model_name}:")
    print(f"- Точность: {accuracy:.4f}")
    print(f"- Время обучения: {train_time:.4f} сек")
    print(f"- Время предсказания: {predict_time:.4f} сек")

#### 3.1.2 Сравнение основных параметров

In [183]:
models = [
    (SVC(kernel='linear', C=0.01, tol=1e-2, random_state=14), "SVC simple"),
    (SVC(kernel='rbf', C=100.0, gamma=10.0, tol=1e-5, random_state=14), "SVC precise"),
    (SVC(kernel='rbf', C=1.0, gamma='scale', tol=1e-3, random_state=14), "SVC balanced")
]

print("Нормализованные данные:")
for model in models:
    evaluate_model(model[0], X_train, X_test, y_train, y_test, model[1])

print("\nНе нормализованные данные:")
for model in models:
    evaluate_model(model[0], X_train_orig, X_test_orig, y_train_orig, y_test_orig, model[1])
    

Нормализованные данные:
SVC simple:
- Точность: 0.2333
- Время обучения: 0.0030 сек
- Время предсказания: 0.0006 сек
SVC precise:
- Точность: 0.9667
- Время обучения: 0.0027 сек
- Время предсказания: 0.0005 сек
SVC balanced:
- Точность: 0.9667
- Время обучения: 0.0025 сек
- Время предсказания: 0.0005 сек

Не нормализованные данные:
SVC simple:
- Точность: 0.9667
- Время обучения: 0.0038 сек
- Время предсказания: 0.0005 сек
SVC precise:
- Точность: 0.9000
- Время обучения: 0.0031 сек
- Время предсказания: 0.0007 сек
SVC balanced:
- Точность: 0.9667
- Время обучения: 0.0026 сек
- Время предсказания: 0.0009 сек


#### 3.1.3 Наблюдения (нормализованные данные):
- Точность сверхпростой модели значительно меньше точности сверхсложной и сбалансированной 
- При продолжительных запусках замеров время предсказания хоть и колеблется, но не сильно отличается между моделями
- Время обучения сильно колеблется между запусками, что не дает сделать однозначных выводов
#### 3.1.4 Наблюдения (оригинальные данные):
- Точность сверхсложной модели уступает точности сверхпростой и сбалансированной
- Точность сверхпростой модели равна точности сбалансированной
- В большинстве запусков тренировка сложной модели занимает больше всего времени
- Из-за сильного колебания между запусками невозможно сравнить время обучения сбалансированной и сверхпростой модели
- Аналогичная ситуация с веременем предсказания: сверхсложная модель показывает наихудший результат, временные затраты на предсказания сверхпростой и сбалансированной моделью не поддаются сравнению из-за постоянного изменения
#### 3.1.5 Наблюдения (между типами данных)
- При нормализации данных сильно страдает точность сверхпростой модели
- Сверхсложная модель работает значительно хуже с ненормализованными данными
- Остальные параметры также сильно отличаются между запусками
#### 3.1.6 Анализ наблюдений:
- Точность сверхпростой модели значительно меньше при использовании нормализовнных данных из-за **сильной регуляризации** - модель становится слишком простой
- Время обучения колеблется из-за **малого размера датасета** (150) - при таком маленьком времени влияние внешних процессов становится слишком значительным
- Точность сверхсложной модели при использовании ненормализованных данных падает из-за **переобучения** - модель слишком подстраивается под шум в данных
- Сверхпростая модель показывает высокую точность при использовании ненормализованных данных, так как **линейная модель лучше работает с естественными масштабами данных**
- Сверхпростая модель страдает при нормализации потому что **C=0.01 слишком чувствителен к изменению формата**
#### 3.1.7 Выводы:
- **Нормализация критически важна для сверхсложных моделей**, но **вредна для сильно регуляризованных линейных моделей**
- **Гиперпараметры должны подбираться под конкретный масштаб данных**
- **На малых датасетах время обучения и предсказания не являются надежными метриками для сравнения моделей**

### 3.2 Визуализация датасета
#### 3.2.1 Уменьшение размерности датасета

In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

best_model = models[2][0]
y_pred_full = best_model.predict(X)

#### 3.2.2 График с выборкой типа из оригинальных данных

In [ ]:
df_viz = pd.DataFrame({
    'PCA1': X_pca[:, 0],
    'PCA2': X_pca[:, 1],
    'original_class': [iris.target_names[i] for i in y]
})

original_graph = px.scatter(
    df_viz,
    x='PCA1',
    y='PCA2',
    color='original_class',
    title='PCA: Iris type from original data',
    labels={'PCA1': 'PCA: Main component 1', 'PCA2': 'PCA: Main component 2'},
    hover_data=['original_class']
)

original_graph.update_traces(marker=dict(size=9, opacity=0.7))
original_graph.show()

#### 3.2.3 График с выборкой типа на основе предсказаний сбалансированной модели

In [73]:
df_viz['predicted_class'] = [iris.target_names[i] for i in y_pred_full]

graph_predicted = px.scatter(
    df_viz,
    x='PCA1',
    y='PCA2',
    color='predicted_class',
    title='PCA: Iris type from predicted data',
    labels={'PCA1': 'Main component 1', 'PCA2': 'Main component 2'},
    hover_data=['predicted_class']
)

graph_predicted.update_traces(marker=dict(size=8, opacity=0.7))
graph_predicted.show()

#### 3.2.4 Наблюдения:
- Предсказания сбалансированной модели оказались достаточно точными, чтобы погрешность не была заметна при визуализации